In [1]:
import gym


#定义环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        env = gym.make('Pendulum-v1')
        super().__init__(env)
        self.env = env

    def reset(self):
        state, _ = self.env.reset()
        return state

    def step(self, action):
        state, reward, done, _, info = self.env.step(action)
        return state, reward, done, info


env = MyWrapper()

env.reset()

array([-0.7696618 , -0.6384518 ,  0.22815812], dtype=float32)

In [12]:
import gym
env = gym.make("LunarLander-v2", render_mode="human")
env.action_space.seed(42)

observation, info = env.reset(seed=42)

for _ in range(1000):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    if terminated or truncated:
        observation, info = env.reset()

env.close()

c:\ProgramData\Anaconda3\envs\RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [2]:
import gym
from stable_baselines3 import PPO

#训练一个模型
model = PPO('MlpPolicy', env, verbose=0).learn(8000, progress_bar=True)

#保存模型
model.save('models/save')

#加载模型
model = PPO.load('models/save')

model

Output()

c:\ProgramData\Anaconda3\envs\RL\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: 
`np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):

c:\ProgramData\Anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [2]:
#加载模型
model = PPO.load('models/save', verbose=0)

#如果要继续训练,需要重新给它一个env,因为env在保存模型时是不能保存下来的
model.set_env(env)

#继续训练
model.learn(8000, progress_bar=True)

NameError: name 'PPO' is not defined

In [2]:
import gym
import numpy as np

# 测试一个环境
def test(env, wrap_action_in_list=False):
    print(env)

    state = env.reset()
    over = False
    step = 0

    while not over:
        action = env.action_space.sample()

        if wrap_action_in_list:
            action = action

        next_state, reward, over,_ = env.step(action)

        if step % 20 == 0:
            print(step, state, action, reward)

        if step > 200:
            break
        state = next_state
        step += 1

test(env)

<MyWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>>
0 [0.07218624 0.99739116 0.8919712 ] [1.7378713] -2.3282253153208705
20 [-0.2385203 -0.9711375 -1.170521 ] [-1.1149726] -3.4202881012693864
40 [-0.55694973  0.8305462   2.9121382 ] [-1.5324578] -5.5225072331571035
60 [-0.984551   -0.17509787 -4.571843  ] [0.6415151] -10.88529545283633
80 [-0.9633814  -0.26813474  3.9805312 ] [0.21465972] -9.822191331867616
100 [-0.6252558  0.7804199 -3.279739 ] [-1.7166805] -6.124280065923935
120 [-0.18435736 -0.98285925  2.4722273 ] [1.0108101] -3.696501197218173
140 [-0.42587793  0.9047806  -3.044164  ] [-0.9456013] -4.970615957293665
160 [-0.15890619 -0.98729366  2.3169239 ] [-0.47413313] -3.53124982863186
180 [ 0.25435236  0.96711165 -2.3977344 ] [-1.6481655] -2.303222614544829
200 [-0.11203683 -0.9937041   3.6140094 ] [0.53798527] -4.139116616997664


In [8]:
# 修改最大步数
class StepLimitWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.current_step = 0
    
    def reset(self):
        self.current_step = 0
        return self.env.reset()

    def step(self, action):
        self.current_step += 1
        state, reward, done, info = self.env.step(action)
        
        # 修改done字段
        if self.current_step >= 100:
            done = True
        
        return state, reward, done, info

test(StepLimitWrapper(env))

<StepLimitWrapper<MyWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>>>
0 [-0.9987958  -0.04906001 -0.0384343 ] [0.16010138] -9.563809677068514
20 [-0.99414724  0.10803361  0.42362726] [1.9890343] -9.223101039549428
40 [-0.99859345  0.05301971 -1.004939  ] [-1.1963421] -9.64155064102953
60 [-0.92510855 -0.37970266  1.4664237 ] [1.6687284] -7.791976793563201
80 [-0.812863    0.5824549  -0.16073538] [-1.1355683] -6.353502565970594


In [4]:
# 修改环境的动作空间
import numpy as np

class NormalizeActionWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        # 获取动作空间
        action_space = env.action_space

        # 动作空间必须是连续值
        assert isinstance(action_space, gym.spaces.Box)

        # 重新定义动作空间，在 -1,+1之间的连续值
        # 这里其实只影响env.action_space.sample的返回结果
        # 实际计算时，还是正负2之间计算的
        env.action_space = gym.spaces.Box(low=-1, high=1,shape=action_space.shape,dtype=np.float32)

        
    def reset(self):
        return self.env.reset()

    def step(self, action):
        # 重新缩放动作的值域
        action = action * 2.0

        action = min(action, 2.0)
        action = max(action, -2.0)

        return self.env.step(action)

test(NormalizeActionWrapper(env))

<NormalizeActionWrapper<MyWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>>>
0 [ 0.82443213 -0.5659608   0.64134234] [0.35957316] -0.40356963721489625
20 [-0.44379258  0.89612955 -6.6596518 ] [0.76076335] -8.560839787785925
40 [-0.9974366   0.07155585  6.7546034 ] [0.26507068] -13.987498185926063
60 [ 0.5134467  -0.85812145 -2.550399  ] [-0.68042195] -1.7165035126270012
80 [ 0.81502736  0.5794225  -0.95199615] [-0.7183617] -0.474642456584224
100 [-0.30434722 -0.95256114  6.379881  ] [0.8711266] -7.6079090172004875
120 [ 0.6894555  -0.72432804 -2.7196856 ] [0.9247283] -1.3992853754728931
140 [ 0.75123686  0.6600327  -1.3111333 ] [0.25518644] -0.6918099804453607
160 [ 0.33620146 -0.9417901   4.513786  ] [-0.09290681] -3.5452375840404002
180 [-0.02041132 -0.9997917  -4.9825    ] [0.35290998] -5.014975330051451
200 [0.6653459  0.74653524 0.72290903] [0.04104136] -0.7626445313694492


In [4]:
import gym
import numpy as np


# 修改状态空间
class StateStepWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        # 状态空间必须是连续值
        assert isinstance(env.observation_space, gym.spaces.Box)

        # 增加一个新状态字段
        low = np.concatenate([env.observation_space.low, [0.0]])
        high = np.concatenate([env.observation_space.high, [1.0]])

        env.observation_space = gym.spaces.Box(low=low, high=high, dtype=np.float32)

        self.step_current = 0
    def reset(self):
        self.step_current = 0
        return np.concatenate([self.env.reset(), [0.0]])

    def step(self, action):
        self.step_current += 1
        state, reward, done, info = self.env.step(action)
        
        return self.get_state(state), reward, done, info
    
    def get_state(self, state):
        # 添加一个新的字段
        state_step = self.step_current / 100
        return np.concatenate([state, [state_step]]) # 加一列

test(StateStepWrapper(env))

<StateStepWrapper<MyWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>>>
0 [-0.08650355 -0.99625152 -0.19312151  0.        ] [0.33387163] -2.75084274915039
20 [-0.47532126  0.8798123   1.78159261  0.2       ] [1.1249055] -4.587547091643162
40 [-0.62708122 -0.77895391 -3.54702616  0.4       ] [1.2109034] -6.315793315379384
60 [-0.98780531  0.15569402  3.87362218  0.6       ] [1.416071] -10.41429326941321
80 [-0.94566798  0.32513383 -4.14462137  0.8       ] [0.353936] -9.616483504146965
100 [-0.6705361  -0.7418769   3.86490726  1.        ] [-1.5554457] -6.812549539032869
120 [-0.53836179  0.84271383 -2.25301814  1.2       ] [-0.7851366] -5.084779853345219
140 [-0.53221834 -0.84660715  1.04854572  1.4       ] [-0.77876776] -4.656039144513375
160 [-0.66517442  0.74668801  1.29779148  1.6       ] [-1.647633] -5.454352129905857
180 [-0.79891515 -0.60144377 -2.9577992   1.8       ] [0.35013768] -7.106422011578631
200 [-0.98384923  0.17899917  4.81690931  2.       

In [9]:
import gym

class Pendulum(gym.Wrapper):
    def __init__(self):
        env = gym.make("Pendulum-v1")
        super().__init__(env)
        self.env = env
    
    def reset(self):
        state, _ = self.env.reset()
        return state

    def step(self, action):
        state, reward, done, _, info = self.env.step(action)
        return state, reward, done, info

Pendulum().reset()

array([-0.49796903,  0.86719483, -0.9840721 ], dtype=float32)

In [10]:
# SB3内置的Wrapper:Monitor
# 训练中增加一些日志
from stable_baselines3 import A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

env = DummyVecEnv([lambda: Monitor(Pendulum())])


A2C('MlpPolicy', env, verbose=1).learn(1000)

Using cuda device
------------------------------------
| time/                 |          |
|    fps                | 115      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.44    |
|    explained_variance | -0.0431  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -37.5    |
|    std                | 1.02     |
|    value_loss         | 832      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 182      |
|    iterations         | 200      |
|    time_elapsed       | 5        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.44    |
|    explained_variance | 0.000636 |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss    

In [11]:
# SB3内置的Warpper
from stable_baselines3.common.vec_env import VecNormalize, VecFrameStack,DummyVecEnv

# VecNormalize,他会对state和reward进行Normalize
env = DummyVecEnv([Pendulum])
env = VecNormalize(env)

test(env)

IndexError: invalid index to scalar variable.